# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/root/miniconda3/envs/ai/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/envs/ai/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/root/miniconda3/envs/ai/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是《易经》中的第一卦，也是八卦之一。乾卦是由两个阴爻夹一个阳爻构成，象征着天、阳气、刚强、积极向上等含义。乾卦的卦辞是“元、亨、利、贞”，表示这个卦具有正始、通達、吉祥、正當的特质。

乾卦的六爻分别代表着不同的含义：

1. 初爻：对应阳爻，表示天、阳气、新生等。
2. 二爻：对应阴爻，表示地、阴暗、柔顺等。
3. 三爻：对应阳爻，表示天、阳光、刚强等。
4. 四爻：对应阴爻，表示地、阴暗、柔软等。
5. 五爻：对应阳爻，表示天、阳光、正义等。
6. 六爻：对应阴爻，表示地、阴暗、静止等。

乾卦的变化规律是：由初爻到三爻，由阳爻到阴爻；由三爻到四爻，由阴爻到阳爻。这种变化代表了中国古代哲学中的“阴阳”观念，即阴阳相互依存、相互转化。乾卦的这种变化规律也象征着天地之间的交互作用，以及万物运动的规律。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是《易经》中的第五卦，也是八卦之一。讼卦由两个阳爻夹一个阴爻构成，象征着诉讼、争端、矛盾等含义。讼卦的卦辞是“损、贞”，表示这个卦具有损耗、和谐、正当的特质。

讼卦的六爻分别代表着不同的含义：

1. 初爻：对应阳爻，表示初起、进展。
2. 二爻：对应阳爻，表示发展、壮大。
3. 三爻：对应阳爻，表示止戈、和平。
4. 四爻：对应阳爻，表示决断、胜利。
5. 五爻：对应阴爻，表示反悔、失败。
6. 六爻：对应阳爻，表示定分、解决。

讼卦的变化规律是：由初爻到三爻，由阳爻到阳爻；由三爻到四爻，由阳爻到阴爻。这种变化代表了中国古代哲学中的“阴阳”观念，即阴阳相互依存、相互转化。讼卦的这种变化规律也象征着事物发展的规律，以及解决矛盾、处理争端的方法。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
#timestamp = "20240225_222843"
timestamp = "20250109_102922"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [10]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [11]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250109_102922）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [12]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250109_102922）：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着君子之交游，预示着君子们互相信任、互相辅助，然而仍需警惕慎行。讼卦说明，尽管地载着水，但水却不得安宁，君子们应当避免纠纷，不了了之。因此，君子应当避免纷争，谨慎行事，以避免冲突。


In [13]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250109_102922）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [14]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [15]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是八卦中的第一个卦象，代表着天，由六个阳爻组成，象征着刚健的特性。在卜卦中，乾卦预示着先祖们创业立国，预示着我们可以在生活中充满活力和自信。在命运中，乾卦代表着刚强的精神，对于事业、事业阶段和决断力的影响都是正面的。

乾卦的卦象为：乾为天，故称为乾。其卦象为纯阳，代表着刚健的特性。在命理学中，纯阳的卦象会带来积极的能量，有助于实现目标和愿望。在解卦中，乾卦表示刚健的特性，预示着先祖们创业立国，对于事业、事业阶段和决断力的影响都是正面的。

乾卦的哲学含义是：天行健，君子以自强不息。这意味着君子应该效仿天的行健，即不断自强，自强不息，从而达到刚健的境地。

乾卦的取象于天，象义为天。其卦阴阳比例均衡，代表着刚健的特性。在命理学中，纯阳的卦象会带来积极的能量，有助于实现目标和愿望。

在解卦中，乾卦表示刚健的特性，预示着先祖们创业立国，对于事业、事业阶段和决断力的影响都是正面的。

乾卦的预测是：先祖们创业立国，对于事业、事业阶段和决断力的影响都是正面的。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 乾卦是八卦中的第一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。乾卦的卦辞是“元、亨、利、贞”，预示着吉祥如意，同时也教导人们要刚健强劲，积极向上，积极进取。在卜卦中，乾卦预示着先王刚健强劲，王道吉祥如意，人民充满力量，事业大功告成。

在周易中，乾卦代表着天，代表着刚健强劲的特性，象征着吉祥如意，事业大功告成。同时，它也预示着人们要刚健强劲，积极向上，积极进取。在卜卦中，人们应该寻求刚健强劲的力量，以预测先知的吉祥之兆。


In [16]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
地水师卦是易经卦象中的一种，由地水卦和乾卦相组合而成。在《易经》中，天地生万物，其中以乾卦为天，地生万物，其中以地水卦为地。地水师卦象征着地的力量，以及天降下水的景象。它象征着众口所归的刚德，以及地中藏着一股形如水的潜力量。

在解卦中，地水师卦被认为是一种吉象，表明刚中有柔，有深厚的德行和广大的刚德。在square event（ square event），地水师卦预示着众口所归的刚德，将会形成一种以柔克刚的局面。在 square matter（ square matter），天时地利，地水师卦预示着刚中有柔，以柔克刚的景象。以柔克刚的策略可以得到众人的支持，从而达到共同的目标。

地水师卦的核心哲学是：以柔克刚，以德服人。在处理问题时，以柔和的手段来克服刚强的对手，以智慧和德行来感化他人，从而达到成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？ 地水师卦是中国古代的一卦，由地水二字组成，取象大地之形，由湖海之水归流大海，象征海纳百川，聚众成塔之象。这一卦寓意着天地大地的形变，预示着时局变化复杂，需要谨慎对待。

在风水学中，地水师卦被认为是一种吉祥的徵兆，象征着团结和聚首，预示着好运即将来临。在解卦中，地水师卦被解释为：

初六：勿相倍舍，取舍以时。

六二：德行未艾，未济何待。

六三：未果其谋，尚可期待。

六四：反复无定，未果其谋。

六五：天未绝人，何忧何虑。

初七：反复无定，未果其谋。

在八字命理学中，地水师卦表示运程波折，事业难成，需谨慎行事。在事业、财富、婚姻等方面可能会面临较大的变数，需保持冷静，不可盲目决策。

在八字中，地水师卦代表的是地支中的辰、未、戌、亥，代表土形，土代表信义，但信义难以坚持，所以地水师卦的总体象征是：好运即将来临，但需要谨慎行事，以土代表信义，保持冷静，不可盲目决策。


In [17]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦（卦名：讼）是一卦，《易经》中的卦象为雷声震天，大象雷声震天，代表天雷CP，初始卦象为雷声震天，时行卦象为天电大行。讼卦象征讼事（诉讼），初始卦象为天雷震天，时行卦象为天电大行，预示着初始可能会有惊cloud（惊恐）之象，而时行时止则有天电大行之象。在讼卦中，的天雷震天表示阳雷，预示着 Initial events in the lawsuit may be very intense, and the court may rule in favor of the plaintiff. However, the court may also be very strict and require the plaintiff to provide evidence, which may delay the case.

讼卦的卦象表示 initial events in the lawsuit may be very intense, and the court may rule in favor of the plaintiff. However, the court may also be very strict and require the plaintiff to provide evidence, which may delay the case. The court may also order the parties to attend court frequently, which may not be suitable for the parties. In the end, the case may be resolved through compromise or judgment, but the outcome is not necessarily favorable to either party.

In the讼卦, the initial events in the lawsuit may be very intense, and the court may rule in favor of the plaintiff. However, the court may also be 

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [19]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [20]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是八卦中的第一个卦象，代表天，由六个阳爻组成，象征刚健 Strengths:

- The initial hexagram represents the天, which is a symbol of strength and power.
- It signifies courage, courage, and determination.
- It indicates a strong will and determination to achieve success.

- The initial line represents the天, and the following lines represent the sun.
- The sun represents prosperity, strength, and power.
- It signifies a time of growth, success, and victory.

- The line in the middle represents the balance of the opposites, and the following lines represent the decline of the sun.
- This stage marks a period of balance and harmony between the forces of darkness and light.

- The final line represents the decline of the sun, and the following lines represent the rise of the moon.
- The moon represents mystery, introspection, and emotional balance.
- It indicates a time of rest and contemplation, and a need for caution and awareness of one's surroundings.

Interpretation:

The乾卦 represents the stren

In [21]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
地水师卦，又称地水卦，是易经卦象中的一种，由地水卦（乾卦下，坤卦上）组合而成。在《易经》中，每一卦都有其特定的象征意义，而地水卦则象征着天地自然的力量，以及人类在这种力量面前的作为。

地水卦的核心哲学是：柔能胜刚，静能生躁。这一卦所传达的信息是，虽然刚柔相激，刚强相持，然而，静止的德行能够生发躁乱的景象。在卜卦时，地水卦预示着初始的胜利，然而，随之而来的是柔弱的胜利。因此，地水卦启示人们要刚强有义，能够坚持柔弱的德行，能够等待时机的到来。

地水卦的取象于山水，其运行趋势是：初建国，次渫 SwEEP，次永广，末制命。在卜卦时，地水卦表示 Initial胜利，然而随着时间的推移，将会有更多的挑战和困难出现。因此，在解卦时，需要警惕 Initial胜利，并做好更多的准备。

总之，地水卦是《易经》中的一卦，其特


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 地水师卦象

 地水师卦象是指由《易经》卦象组成的牌阵，由乾、坤、坎、离四卦组合而成，象数学认为，坎代表水，离代表火，乾代表天，坤代表地，地水师卦象因此具有化天化地的特点。

在命理学中，地水师卦象被认为是一种吉祥的牌阵，可以带来田产、财富、荣誉和地位等方面的好运。当然，这需要根据个人的命盘和解析来判断。地水师卦象的优点在于能够带来田产、财富、荣誉和地位等方面的好运，但缺点在于可能会带来一些负面的影响，如疾病、灾祸等。

需要注意的是，卦象只是古代哲学和命理学中的一种理论工具，不能完全决定一个人的命运。在现实生活中，一个人的命运取决于多种因素，如努力、机遇、人际关系等。因此，不要过分迷信卦象，而应该以实际生活中积极的态度和努力去创造美好未来。


In [22]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：天水讼卦

原始输出：
的天水讼卦，是易经中的卦象之一，由乾卦和坎卦组成，象征着天和水的特性。在卜筮时，如果得到这个卦象，意味着好运和困境并存，需要谨慎面对，不可盲目和使用力量。

在解卦时，需要认为天水讼卦预示着先吉利后艰难，先平静后ond ark，利见大人。在决策时，应该慢view病情，不可草率行事，等待时机成熟。

天猫天使社区


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 天水讼卦

天山童子

坎卦

坎字有形有象，象征水。坎卦为泽卦，由水组成，象征恩惠和柔顺。卦中代表卦象的由下到上分别为：水-形-象，代表由下到上的变化。

坎卦的特性为流动，代表变化和适应，同时代表柔顺和耐心。在占卜中，坎卦预示着文书、法律和消息，同时强调brand（品牌）和message（信息）。在商业中，代表着市场和财富，同时预示着风险和 soft power（软实力）。

在运势方面，坎卦预示着顺利，具有柔顺和耐心的特点。然而，过于柔顺可能会导致优柔寡断。在人际关系方面，代表着和谐和团结，但需要注意避免沉溺于感情，而应保持理智。

坎卦的解卦为形上形下，解卦为上卦坎，下卦乾（天）。解卦表示解救、和解和托付的意思，预示着好运和成功。

坎卦的卦象为天（乾）下，水（坎）上，形成形上下卦。卦象为形上形下，解卦为上卦坎，下卦乾（天）。解卦表示解救、和解和托付的意思，预示着好运和成功。

综上所述，坎卦具有流动、变化和适应的特点，同时代表着柔顺、耐心和智慧。在商业和市场方面，代表着财富、风险和软实力。在人际关系和运势方面，预示着和谐、团结和理智。
